In [1]:
print('hello world')

hello world


In [4]:
words = open('../names.txt', 'r').read().splitlines()
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [5]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
uniq=list(set(c for w in words for c in w))
uniq= uniq + ['.']
vocab=sorted(uniq)
stoi = {c:i for i,c in enumerate(vocab) }
itos = {i:s for s,i in stoi.items()}

In [35]:
vocab_len=len(vocab)
vocab_len

27

In [ ]:
block_size=3
X,Y=[],[]
for w in words[:5]:
  context = [0]*block_size
  for ch in w+'.':
    X.append(context)
    Y.append(stoi[ch])
    print(f'{"".join(itos[c] for c in context)} --> {ch}')
    context = context[1:] + [stoi[ch]]

X=torch.tensor(X)
Y=torch.tensor(Y)

... --> e
..e --> m
.em --> m
emm --> a
mma --> .
... --> o
..o --> l
.ol --> i
oli --> v
liv --> i
ivi --> a
via --> .
... --> a
..a --> v
.av --> a
ava --> .
... --> i
..i --> s
.is --> a
isa --> b
sab --> e
abe --> l
bel --> l
ell --> a
lla --> .
... --> s
..s --> o
.so --> p
sop --> h
oph --> i
phi --> a
hia --> .


In [52]:
X.shape,Y.shape


(torch.Size([32, 3]), torch.Size([32]))

In [72]:
n_emb=2
C=torch.randn((vocab_len, 2))
C.shape

torch.Size([27, 2])

In [73]:
emb=C[X]
print(emb.shape)

emb[3], X[3], C[[5,13,13]]

torch.Size([32, 3, 2])


(tensor([[0.0904, 0.2052],
         [0.0442, 1.8832],
         [0.0442, 1.8832]]),
 tensor([ 5, 13, 13]),
 tensor([[0.0904, 0.2052],
         [0.0442, 1.8832],
         [0.0442, 1.8832]]))

In [ ]:
n_neurons=100
W1 = torch.randn((block_size*n_emb, n_neurons))
b1 = torch.randn(n_neurons)
W1.shape, b1.shape



(torch.Size([6, 100]), torch.Size([100]))

In [77]:
W2 = torch.randn((n_neurons, vocab_len))
b2 = torch.randn(vocab_len)
W2.shape, b2.shape

(torch.Size([100, 27]), torch.Size([27]))

In [ ]:
preact = emb.view(-1, block_size*n_emb) @ W1 + b1
h = torch.tanh(preact)
logits = h @ W2 + b2
probs =  F.softmax(logits, 1)

In [117]:
loss = F.cross_entropy(logits, Y)
loss

tensor(14.8941)

In [ ]:
# train prep
n_emb=2
n_neurons=100
block_size=3

X,Y=[],[]
for w in words:
  context = [0]*block_size
  for ch in w+'.':
    X.append(context)
    Y.append(stoi[ch])
    #print(f'{"".join(itos[c] for c in context)} --> {ch}')
    context = context[1:] + [stoi[ch]]

X=torch.tensor(X)
Y=torch.tensor(Y)

# init
C  = torch.randn((vocab_len, n_emb))
W1 = torch.randn((block_size*n_emb, n_neurons))
b1 = torch.randn(n_neurons)
W2 = torch.randn((n_neurons, vocab_len))
b2 = torch.randn(vocab_len)

parameters = [C,W1,b1,W2,b2]
for p in parameters:
  p.requires_grad = True

In [126]:
X.shape, Y.shape

(torch.Size([228146, 3]), torch.Size([228146]))

In [ ]:
# train
lr = 0.01

for _ in range(10_000):
  # minibatch
  batch = torch.randint(0, X.shape[0], (64,))

  # forward pass
  emb=C[X[batch]]
  preact = emb.view(-1, block_size*n_emb) @ W1 + b1
  h = torch.tanh(preact)
  logits = h @ W2 + b2
  loss = F.cross_entropy(logits, Y[batch])
  
  # clear grads
  for p in parameters:
    p.grad = None
  # recompute grads
  loss.backward()
  # update
  for p in parameters:
    p.data += -lr * p.grad

loss

tensor(2.3073, grad_fn=<NllLossBackward0>)

In [ ]:
# inference
out=[]
for i in range(5):
  out.append([])
  context = [0] * block_size
  while True:
    # forward pass
    emb = C[context]
    preact = emb.view(-1, block_size*n_emb) @ W1 + b1
    h = torch.tanh(preact)
    logits = h @ W2 + b2
    probs = F.softmax(logits, 1)
    hit = torch.multinomial(probs, num_samples=1, replacement=True).item()
    out[i].append(hit)
    if hit == 0:
      break
    context = context[1:] + [hit]
for o in out:
  print(''.join(itos[x] for x in o))

setcymone.
korboreya.
noraj.
korand.
eloyd.
